In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import solve, norm, cond
from scipy.linalg import hilbert

In [2]:
def lu(a): #алгоритм LU-декомпозиции
    n = a.shape[0]
    l = np.identity(n)
    u = np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            if i <= j:
                u[i,j] = a[i,j]-sum([l[i,k]*u[k,j] for k in range(i)])
            else:
                l[i,j] = (a[i,j]-sum([l[i,k]*u[k,j] for k in range(j)]))/u[j,j]
    return l,u

In [3]:
def lu_solve(l,u,b=None): #решение СЛАУ LU-методом
    if b is None:
        b = np.random.uniform(-100,100,size=(u.shape[1]))
    y = np.zeros(l.shape[1])
    x = np.zeros(u.shape[1])
    n = len(x)
    for i in range(len(y)):
        y[i] = b[i] - sum([l[i,p]*y[p] for p in range(i)])
    for j in range(len(x)):
        x[n-j-1]=(y[n-j-1]-sum([u[n-j-1,n-p-1]*x[n-p-1] for p in range(j)]))/u[n-j-1,n-j-1]
    return x
            

Проверим, что решения, полученные lu-методом, совпадают с решением системы:

In [13]:
a = np.random.rand(10,10)
b = np.random.rand(10)
l,u = lu(a)
norm(lu_solve(l,u,b)-solve(a,b))

1.5795245481498015e-15

In [5]:
matrixes = [hilbert(n) for n in range(3,6)]

In [8]:
def regularisation_solution(a,b=None):
    if b is None:
        b = np.random.uniform(-100,100,size=(a.shape[1]))
    ans = pd.DataFrame(columns=["alpha","cond(a+alpha*E)","||x-x_alpha||","||b-A*x_alpha||"])
    l,u = lu(a)
    x = lu_solve(l,u,b)
    ans = ans.append(pd.Series([0,cond(a),x,norm(b-a@x)],index=ans.columns),True)
    E = np.identity(a.shape[0])
    x = solve(a,b)
    for i in range(2,13,2):
        a_i = a + 10**(-i)*E
        l,u = lu(a_i)
        x_i = lu_solve(l,u,b)
        ans = ans.append(pd.Series([10**(-i),cond(a_i),norm(x_i-x),norm(b-a@x_i)],index=ans.columns),True)
    return ans

In [9]:
regularisation_solution(matrixes[0],np.array([1,1,1])) #результат для матрицы Гильберта 3-го порядка

,alpha,cond(a+alpha*E),||x-x_alpha||,||b-A*x_alpha||
0,0.000000e+00,524.056778,0.000000e+00,0.000000e+00
1,1.000000e-02,111.790091,3.009003e+01,9.450946e-02
2,1.000000e-04,505.291334,1.369522e+00,3.717896e-03
3,1.000000e-06,523.862213,1.419955e-02,3.852163e-05
4,1.000000e-08,524.054831,1.420478e-04,3.853556e-07
5,1.000000e-10,524.056758,1.420483e-06,3.853570e-09
6,1.000000e-12,524.056777,1.420498e-08,3.853539e-11


In [21]:
regularisation_solution(matrixes[1],np.array([1,1,1,1])) #результат для матрицы Гильберта 4-го порядка

,alpha,cond(a+alpha*E),||x-x_alpha||,||b-A*x_alpha||
0,0.000000e+00,15513.738739,0.000000,5.024296e-15
1,1.000000e-02,149.575003,232.609406,1.171839e-01
2,1.000000e-04,7627.334553,119.130323,1.181379e-02
3,1.000000e-06,15354.963172,2.398420,2.334470e-04
4,1.000000e-08,15512.134730,0.024230,2.358064e-06
5,1.000000e-10,15513.722697,0.000242,2.358303e-08
6,1.000000e-12,15513.738579,0.000002,2.358282e-10


In [22]:
regularisation_solution(hilbert(5),hilbert(5)@np.ones(5)) #результат для матрицы Гильберта 5-го порядка

,alpha,cond(a+alpha*E),||x-x_alpha||,||b-A*x_alpha||
0,0.000000e+00,476607.250242,0.000000e+00,5.087681e-16
1,1.000000e-02,157.653234,1.300612e-01,2.195547e-02
2,1.000000e-04,15172.641273,1.167345e-02,2.235339e-04
3,1.000000e-06,365456.558250,1.104925e-03,2.236058e-06
4,1.000000e-08,475162.081827,1.431873e-05,2.236068e-08
5,1.000000e-10,476592.755044,1.436282e-07,2.236067e-10
6,1.000000e-12,476607.105284,1.457316e-09,2.236068e-12
